In [1]:
import numpy as np
import timeit
from tqdm.auto import tqdm
import pandas as pd

def check_correctness(eigvals_func, A, rtol=1e-2, atol=1e-3):
    assert np.allclose(sorted(np.linalg.eigvals(A)), sorted(eigvals_func(A)), rtol=rtol, atol=atol)

def make_experiment(eigvals_func, N=50, random_seed=0, correctness_tests=10, rtol=1e-2, atol=1e-3, estimated_time=10, retries=7):
    np.random.seed(random_seed)
    for _ in tqdm(range(correctness_tests), desc="Checking correctness"):
        A = np.random.randn(N, N)
        A = A.dot(A.T)
        check_correctness(eigvals_func, A, rtol=rtol, atol=atol)
    
    variables = {'eigvals_func': eigvals_func, 'A': A}
    single_execution = timeit.timeit(stmt="eigvals_func(A)", globals=variables, number=1)
    number = max(int(np.floor(estimated_time / single_execution / retries)), 1)
    times = [
        timeit.timeit(stmt="eigvals_func(A)", globals=variables, number=number) / number for _ in range(retries)
    ]
    return times

make_experiment(np.linalg.eigvals)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

[0.0004407746141215106,
 0.0004099273891625616,
 0.00040613379310344813,
 0.0004283136288998355,
 0.00040757142857142864,
 0.00041658292282430203,
 0.0004378585878489328]

In [2]:
n_values = [5, 10, 20, 30, 40, 50]
results = []

## Базовая имплементация на Numpy

In [3]:
def qr(A):
    Q = A.copy()
    Q[:, 0] /= np.linalg.norm(Q[:, 0])
    for i in range(1, A.shape[0]):
        Q[:, i] -= Q[:, :i].dot(Q[:, :i].T).dot(Q[:, i])
        Q[:, i] /= np.linalg.norm(Q[:, i])
    R = Q.T.dot(A)
    return Q, R

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        Q, R = qr(A)
        A = R.dot(Q)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

## Попытка ускорения итерации на Numpy

In [4]:
def make_iteration(A):
    Q = A.copy()
    Q[:, 0] /= np.linalg.norm(Q[:, 0])
    for i in range(1, A.shape[0]):
        Q[:, i] -= Q[:, :i].dot(Q[:, :i].T).dot(Q[:, i])
        Q[:, i] /= np.linalg.norm(Q[:, i])
    return Q.T.dot(A).dot(Q)

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

## Оптимизация вычисления функции на Numpy за счёт кэширования результатов

In [5]:
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return np.linalg.multi_dot([Q, A, Q.T])

def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

## Numba JIT

In [6]:
from numba import jit

@jit(nopython=True)
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return Q.dot(A).dot(Q.T)

@jit(nopython=True)
def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

## Numba JIT с дополнительными параметрами

In [7]:
from numba import jit

@jit(nopython=True, nogil=True, fastmath=True, cache=True)
def make_iteration(A):
    Q = A.T.copy()
    saved_Q = np.zeros_like(A)
    for i in range(A.shape[0]):
        Q[i] -= saved_Q.dot(Q[i])
        Q[i] /= np.linalg.norm(Q[i])
        saved_Q += np.outer(Q[i], Q[i])
    return Q.dot(A).dot(Q.T)

@jit(nopython=True, nogil=True, fastmath=True, cache=True)
def eigenvalues(A, rtol=1e-6, atol=1e-8):
    squared_A = np.square(A)
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        A = make_iteration(A)
        squared_A = np.square(A)
    return np.diag(A)

for n in n_values:
    result = make_experiment(eigenvalues, N=n)
    results.append(result)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

## Numba guvectorize

In [8]:
from numba import guvectorize, float64

@guvectorize(["void(float64[:, ::1], float64, float64, float64[::1])"], "(n,n), (), () -> (n)", nopython=True, target='cpu')
def eigenvalues(A, rtol=1e-6, atol=1e-8, out=None):
    squared_A = A * A
    while squared_A.sum() - np.diag(squared_A).sum() >= rtol * squared_A.sum() + atol:
        Q = A.T.copy()
        Q[0] /= np.linalg.norm(Q[0])
        saved_Q = np.zeros_like(A)
        for i in range(A.shape[0]):
            Q[i] -= saved_Q.dot(Q[i])
            Q[i] /= np.linalg.norm(Q[i])
            saved_Q += np.outer(Q[i], Q[i])
        A = Q.dot(A).dot(Q.T)
        
        squared_A = A * A
    out[:] = np.diag(A)

def eigenvalues_wrapper(A, rtol=1e-6, atol=1e-8):
    result = np.empty(A.shape[0], dtype=float)
    eigenvalues(A, rtol, atol, result)
    return result

for n in n_values:
    result = make_experiment(eigenvalues_wrapper, N=n)
    results.append(result)

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

Checking correctness:   0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
names = ['Numpy 1', 'Numpy 2', 'Numpy 3', 'Numba JIT', 'Numbda JIT с доп. пар.', 'Numba guvectorize']

In [20]:
total_results = np.array(results).reshape(len(names), len(n_values), -1) * 1000

In [45]:
R = pd.DataFrame(np.core.defchararray.add(np.core.defchararray.add(np.round(total_results.mean(axis=-1), 1).astype(str), " ± "), np.round(total_results.std(axis=-1), 1).astype(str)), index=names, columns=n_values)

In [46]:
R

,5,10,20,30,40,50
Numpy 1,1.7 ± 0.0,4.9 ± 0.1,32.6 ± 1.1,145.2 ± 2.1,122.0 ± 1.7,298.5 ± 5.3
Numpy 2,1.7 ± 0.0,4.9 ± 0.2,32.4 ± 0.6,149.4 ± 5.2,117.0 ± 3.7,291.7 ± 3.3
Numpy 3,2.2 ± 0.1,5.6 ± 0.2,32.6 ± 0.9,139.9 ± 7.1,112.1 ± 1.7,273.2 ± 11.5
Numba JIT,0.3 ± 0.0,0.6 ± 0.1,4.2 ± 0.1,25.6 ± 0.8,26.8 ± 1.0,77.1 ± 0.7
Numbda JIT с доп. пар.,0.2 ± 0.0,0.6 ± 0.0,4.3 ± 0.1,24.6 ± 0.2,27.7 ± 2.0,81.1 ± 4.1
Numba guvectorize,0.2 ± 0.0,0.5 ± 0.0,4.5 ± 0.2,26.7 ± 0.8,29.1 ± 0.3,88.8 ± 2.7


In [47]:
for i, j in enumerate(np.round(total_results.mean(axis=-1), 1).argmin(axis=0)):
    R.iloc[j, i] = '\\textbf{' + R.iloc[j, i] + "}"

In [48]:
R

,5,10,20,30,40,50
Numpy 1,1.7 ± 0.0,4.9 ± 0.1,32.6 ± 1.1,145.2 ± 2.1,122.0 ± 1.7,298.5 ± 5.3
Numpy 2,1.7 ± 0.0,4.9 ± 0.2,32.4 ± 0.6,149.4 ± 5.2,117.0 ± 3.7,291.7 ± 3.3
Numpy 3,2.2 ± 0.1,5.6 ± 0.2,32.6 ± 0.9,139.9 ± 7.1,112.1 ± 1.7,273.2 ± 11.5
Numba JIT,0.3 ± 0.0,0.6 ± 0.1,\textbf{4.2 ± 0.1},25.6 ± 0.8,\textbf{26.8 ± 1.0},\textbf{77.1 ± 0.7}
Numbda JIT с доп. пар.,\textbf{0.2 ± 0.0},0.6 ± 0.0,4.3 ± 0.1,\textbf{24.6 ± 0.2},27.7 ± 2.0,81.1 ± 4.1
Numba guvectorize,0.2 ± 0.0,\textbf{0.5 ± 0.0},4.5 ± 0.2,26.7 ± 0.8,29.1 ± 0.3,88.8 ± 2.7


In [49]:
R[np.arange(10, 60, 10)].reset_index().to_csv("../../report/tables/results.csv", sep=';', index=False)